# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git00-
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git00- /tmp/pip-req-build-pfbtr_c6 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git00- /tmp/pip-req-build-pfbtr_c6 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

약 6분 소요

In [3]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

## 4단계 - 잘 실행되는지 확인

약 1분 소요

In [5]:
device = "cuda:0"

messages = [
    {"role": "system", "content": "당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요."},
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

generated_ids = model.generate(model_inputs, max_new_tokens=512, eos_token_id=terminators, do_sample=True, repetition_penalty=1.05,)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

여러분, 은행의 기준 금리, 이를 자주 묻습니다!💸

은행의 기준 금리는 금융기관 중 이자율이 가장 낮아 지지되는 금리를 말합니다. 이는 은행간 거래를 목적으로 정한 이자율로, 모든 은행장 계정의 기준이 되는 금리를 뜻합니다. 해당 국의 중앙은행과 금융 당국이 설정하는 단기금리 및 저금리 대상자들에게 적용됩니다.

기준 금리에는 다음과 같은 유용한 역할이 있습니다.

1. 자금 조달 : 은행들은 기준 금리를 참고하여 연체이자금리인 현금 보유 비용(lending rate)이나 차입비용을 적용함으로써 자금을 조달하거나 빌려줄 수 있습니다.

2. 자산가격 결정 : 기준 금리는 은행의 자산 가격과 주가와도 연관이 있습니다. 기준 금리가 상승하면 연체금리가 상승하게 됩니다,

3. 부동산 등 경제성장: 은행의 기준 금리에 민감하게 반응합니다. 기준 금리의 상승이나 하락은 부동산 시장의 변동에 영향을 미치기도 합니다. 즉, 상승하면 부동산 가격이 오르며 하락하면 부동산 가격이 내립니다

예를 들어, 아마존의 기준 금리가 0% 에서 5% 로 상승한다면, 이 시점에서 은행으로부터 자금을 도급받는 소비자들의 자금 수요도 상승하게 될 수 있습니다.
즉, 기준 금리 인상이 일어나면, 은행은 돈을 빌릴 수 있는 원가가 올라 가게 되고, 결국 상승하는 이자율이 형성될 것입니다.<|end_of_text|>


이번에는 RAG로 실험해보자.

In [6]:
search_result = '''쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 ‘쏘카 비즈니스’ 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 “해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다”며 “전문기관, 업체와의 협력을 통해 새로운 기술을 해군 수송업무에 도입해 해군이 그려나가는 ‘스마트 해군’ 건설에 한 걸음 더 다가갈 수 있도록 노력하겠다”고 말했다. 김남희 쏘카 신규사업본부장은 “일반 기업체 외에도 군이나 지자체, 공공기관 등에서도 법인용 차량 공유 서비스에 대한 수요가 꾸준히 늘어나고 있다”며 “업무 이용 패턴과 특성에 맞는 서비스 인프라와 라인업을 지속해서 확대해 나갈 것”이라고 말했다.'''

user_prompt = '검색 결과:\n' + search_result + '\n\n질문: 해군이 쏘카와 도입하는 서비스는?'

messages = [
    {"role": "system", "content": "당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요."},
    {"role": "user", "content": user_prompt}
]

encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

generated_ids = model.generate(model_inputs, max_new_tokens=512, eos_token_id=terminators, do_sample=True, repetition_penalty=1.05,)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과:
쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 ‘쏘카 비즈니스’ 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 “해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다”며 “전문기관, 업체와의 협력을 통해 새로운 

## 5단계- RAG 시스템 결합하기

In [7]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain_community

In [9]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과: {context}

질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [11]:
from google.colab import drive

# Google 드라이브를 마운트합니다.
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 6.6 MB/s eta 0:00:00


In [34]:
all_documents

[Document(page_content='{"DataSet": "연령대별특징적발화음성", "Version": "1.0", "Date": "20220901", "MediaUrl": "단어/40대/교육/10_13_2066_220901_0105.wav", "DialogID": "10_13_2066_220901_0105", "Category": "40대", "SubCategory": "교육", "DialogPlace": "집", "DialogVoiceType": "독백", "SpeakerNumber": "1", "RecDevice": "스마트폰", "RecLen": 3.585, "AudioResolution": {"BitDepth": "16bit", "SampleRate": "44.1kHz"}, "Speakers": [{"Speaker": "2066", "Gender": "여성", "Locate": "수도권", "AgeGroup": "40"}], "Dialogs": [{"DialogNum": 1, "Speaker": "2066", "SpeakerText": "핵아싸", "TextConvert": "엄청난 아웃사이더", "StartTime": 1.0, "EndTime": 2.585, "SpeakTime": 1.585, "WordInfo": [{"Word": "핵아싸", "WordType": "은어", "WordStructure": "혼성어", "WordDefine": "크다는 뜻의 \'핵\'과 잘 어울려 지내지 못하는 사람을 의미하는 \'아웃사이더\'(Outsider)의 합성어. /  아무도 모르는 존재, 무리에 전혀 어울리지 못하는 존재", "WordFell": "부정", "WordMean": "싫어함"}]}]}', metadata={'source': '/content/drive/MyDrive/02.라벨링데이터/10_13_2066_220901_0105.json'}),
 Document(page_content='{"DataSet": "연령대별특징적발화음성", "Ver

In [50]:
len(all_documents)

25779

In [49]:
# JSON 파일들이 있는 드라이브 내 디렉터리 경로
directory_path = '/content/drive/MyDrive/02.라벨링데이터'

# 모든 JSON 파일을 저장할 리스트
all_documents = []

# 디렉터리 내 모든 파일 리스트
all_files = os.listdir(directory_path)

# JSON 파일 필터링
json_files = [filename for filename in all_files if filename.endswith('.json')]

# 처음 100개의 JSON 파일만 선택
selected_files = json_files[:]

# 선택된 파일에 대해 반복
for filename in selected_files:
    file_path = os.path.join(directory_path, filename)

    # JSON 파일을 UTF-8 인코딩으로 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

        # 문서 내용 디코딩 (필요한 경우)
        page_content = json.dumps(data, ensure_ascii=False)

        # Document 객체로 변환
        doc = Document(page_content=page_content, metadata={"source": file_path})
        all_documents.append(doc)



KeyboardInterrupt: 

In [51]:
#loader = PyPDFLoader("/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
#pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(all_documents)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [52]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [53]:
rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a4a0d317f70>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
}
| LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n검색 결과: {context}\n\n질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7a4b28425f90>))

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
i.metadata

{'source': '/content/drive/MyDrive/02.라벨링데이터/10_13_2070_220901_0003.json'}

In [62]:
result = rag_chain.invoke("귀염뽀짝이 뭐게?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']}  \n\n")

print('--' * 50)
print(f"\n답변: {result['text']}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


주어진 근거: "AgeGroup": "50"}], "Dialogs": [{"DialogNum": 19, "Speaker": "4460", "SpeakerText": "그렇군요, 감사해요. 그러면 귀염뽀짝이 뭔지 알려주실 수 있을까요?", "TextConvert": "그렇군요 감사해요 그러면 귀여운 동물이나 아이돌의 모습을 묘사할 때 쓰는 표현이 뭔지 알려주실 수 있을까요?", "StartTime": 111.33, "EndTime": 116.79, "SpeakTime": 5.46, "WordInfo": [{"Word": "귀염뽀짝", "WordType": "은어", "WordStructure": "혼성어", "WordDefine": "인터넷에서 귀여운 동물이나 아이돌의 모습을 묘사할 때 즐겨 쓰는 표현 / '매우 귀엽다'는 의미", "WordFell": "긍정", "WordMean": "좋음"}]}]} / 출처: /content/drive/MyDrive/02.라벨링데이터/50_51_44605460_221025_0012_19_4460.json  


주어진 근거: "AgeGroup": "50"}], "Dialogs": [{"DialogNum": 20, "Speaker": "5545", "SpeakerText": "귀염뽀짝이 뭐냐면, 귀여운 동물이나 아이돌의 모습을 묘사할 때 쓰는 표현이에요.", "TextConvert": "귀여운 동물이나 아이돌의 모습을 묘사할 때 쓰는 표현이 뭐냐면 귀여운 동물이나 아이돌의 모습을 묘사할 때 쓰는 표현이에요.", "StartTime": 156.68, "EndTime": 163.89, "SpeakTime": 7.21, "WordInfo": [{"Word": "귀염뽀짝", "WordType": "은어", "WordStructure": "혼성어", "WordDefine": "인터넷에서 귀여운 동물이나 아이돌의 모습을 묘사할 때 즐겨 쓰는 표현 / '매우 귀엽다'는 의미", "WordFell": "긍정", "WordMean": "좋음"

In [ ]:
import pickle

# RAG 체인을 pickle 파일로 저장
with open("rag_chain.pkl", "wb") as f:
    pickle.dump(rag_chain, f)


In [ ]:
import pickle

# RAG 체인을 pickle 파일에서 로드
with open("rag_chain.pkl", "rb") as f:
    loaded_rag_chain = pickle.load(f)


In [ ]:
import faiss

# FAISS 인덱스를 파일로 저장
index_path = "/path/to/save/faiss_index"
faiss.write_index(db.index, index_path)

import faiss

# FAISS 인덱스를 파일에서 로드
loaded_index = faiss.read_index(index_path)

from langchain.vectorstores import FAISS

# 로드한 인덱스를 사용하여 검색기 설정
loaded_db = FAISS(embedding_function=hf, index=loaded_index)

# 검색 시스템 설정
loaded_retriever = loaded_db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)
